In [2]:
import os, sys
import pandas as pd
import geopandas
import numpy as np
import urllib.request
from dateutil.parser import parse
from SSURO_soil_data_functions import retrieve_soil_info_by_ID, get_table_headers, get_ssurgo_inventory, impute_data
from fancyimpute import KNN
from math import log
import math

Using TensorFlow backend.


In [3]:
def SaxtonRawls(pSand, pClay, pOM):
    pSand = pSand/100
    pClay = pClay/100
    pOM = pOM/100
    
    # calc LL15 (theta_1500)
    theta_1500t = -0.024*pSand + 0.487*pClay + 0.006*pOM + 0.005*pSand*pOM - 0.013*pClay*pOM + 0.068*pSand*pClay + 0.031
    LL15 = theta_1500t + (0.14*theta_1500t - 0.02)
    LL15 = round(max(0.01, min(0.99, LL15)),3)
    
    # calc DUL (theta_33)
    theta_33t = -0.251*pSand + 0.195*pClay + 0.011*pOM +0.006*pSand*pOM - 0.027*pClay*pOM + 0.452*pSand*pClay + 0.299
    DUL = theta_33t + (1.283*theta_33t**2 - 0.374*theta_33t - 0.015)
    DUL = round(max(0.01, min(0.99, DUL)),3)
    
    # calc SAT-33 KPa moisture
    theta_sat33t = 0.278*pSand + 0.034*pClay + 0.022*pOM - 0.018*pSand*pOM - 0.027*pClay*pOM - 0.584*pSand*pClay + 0.078
    theta_sat33 = theta_sat33t + (0.636*theta_sat33t - 0.107)
    
    # calc SAT
    SAT = DUL + theta_sat33 - 0.097*pSand + 0.043
    SAT = round(max(0.01, min(0.99, SAT)),3)
    
    # calc BD
    BD = (1 - SAT)*2.65
    BD = round(max(1.0, min(2.1, BD)),3)
  
    # calc ksat (saturated water conductivity)
    lambd = (log(DUL)-log(LL15)) / (log(1500)-log(33))
    ksat = 1930*((SAT-DUL)**(3-lambd))
    SWCON = round(0.15 + min(ksat,75)/100, 3)
    
    res = []
    res = [BD, LL15*100, DUL*100, SAT*100, SWCON*100,ksat]
    #names(res) = c("BD", "LL15", "DUL", "SAT", "SWCON","KSAT")
  
    return(res)
    
  


In [33]:
SaxtonRawls(.32,40.5,.25)[5]

1.718408660039206

In [28]:
###Definitions
df = pd.read_csv("/Users/timreeves/Desktop/Python_Stuff/maize_yield_pred/data/Soil_data/OKLAHOMA@JOHNSTON.csv")
def Trunk(row, column_name, firsttrunk, secondtrunk = None):
    if row[column_name] < firsttrunk:
        return firsttrunk
    elif row[column_name] > secondtrunk:
        return secondtrunk
    else:
        return row[column_name]
def Soil_Physical_Prop():
    df["bd"] = df.apply(lambda row: Trunk(row, "dbthirdba", 0.9, 1.8),axis=1)
    df["ksat"] = df.apply(lambda row: 
                          (min(row["ksat"]*100/1.157, 
                               SaxtonRawls(row["sandtotal"] , row["claytotal"], row["om"])[5]*24)), axis=1)
    df["ksat"] = df.apply(lambda row: Trunk(row, "ksat", 0, 500), axis=1)
    df["sat"] = df.apply(lambda row: SaxtonRawls(row["sandtotal"] , row["claytotal"], row["om"])[3]/100, axis=1)
    df["PO"] = df.apply(lambda row: 1-row["bd"]/2.65, axis=1)
    df["Salb"] = 0.15
    df["MWCON"] = 1
    df["dul"] = df.apply(lambda row: row["wthirdba"]/100, axis=1)
    df["dul"].fillna(0, inplace=True)
    df["ll"] = df.apply(lambda row: row["wfifteenba"]/100, axis=1)
    df["ll"].fillna(0, inplace=True)
    df["SWCON"] = df.apply(lambda row: (row["PO"]-row["dul"]/row["PO"]), axis=1)
    df["center"] = df.apply(lambda row: row["hzdept"]+(row["hzthk"]/2), axis=1)
    df["AirDry"] = df.apply(lambda row: 0.9*row["ll"] if row["center"]<=15 else 0.95*row["ll"] if 
                            row["center"]<=30 else 1*row["ll"], axis=1)
    df["U"] = df.apply(lambda row: (5+.175*row["claytotal"]) if row["claytotal"]<=20
                       else (7.5 + 0.05*row["claytotal"]) if row["claytotal"] <=40 
                       else (11.5-.05*row["claytotal"])if row["claytotal"]<=50 
                       else (12.75-0.075*row["claytotal"]) if row["claytotal"]<=70
                       else (11-0.05*row["claytotal"]) if row["claytotal"]<=80
                       else 0, axis=1)
    df["cana"] = df.apply(lambda row: (0.025*row["claytotal"]+3.25) if row["claytotal"]<=30 
                  else (4) if row["claytotal"]<=50 
                  else (-0.025*row["claytotal"]+5.25) if row["claytotal"]<=70 
                  else (3.5) if row["claytotal"]<=80 
                  else 0, axis=1)
    
                       
    
    
    df["DiffusConst"] = 40
    df["DiffusSlope"] = 16
    df["CN2"] = 80
    df["CNRed"] = 20
    df["CNCov"] = 0.8
    df["EnrAcoeff"] = 7.4
    df["EnrBcoeff"] = 0.2
    df["XF_maize"] = 1
    df["KL_maize"] = df.apply(lambda row: .08 if row["center"]<=20 else (.09 * math.exp(-.007*row["center"])), axis=1)
    df["e"] = 0.5
    df["ph"] = df.apply(lambda row: 0.52+1.06*row["ph1to1h2o"], axis=1)
    df["OC"] = df.apply(lambda row: row["om"]/1.72, axis=1)
    ###horizon$OC <- c(horizon$OC[1],
                    #ifelse(horizon$center[-1] >= 100 & diff(horizon$OC) == 0,
                           #horizon$OC[1]*exp(horizon$center[-1]*-0.035),
                           #horizon$OC))
                #This new addition to OC doesn't make much sense to me
                #Indenxing to calculate all rows based off of the first or last?
    df["FInert"] = df.apply(lambda row: 0.4 if row["center"]<=1
                           else 0.4 if row["center"]<=10
                           else .008*row["center"]+.32 if row["center"]<60
                           else .8 if row["center"]<=120
                           else .0032*row["center"]+.42 if row["center"]<180
                           else .99 if row["center"]<=300
                           else 0, axis=1)
    df["FBiom"] = df.apply(lambda row: 0.4 if row["center"]<=10
                           else .055-.0015*row["center"] if row["center"]<=20
                           else .03-.0005*row["center"] if row["center"]<=30
                           else .0216-.0002*row["center"] if row["center"]<60
                           else .01 if row["center"]<300
                           else 0, axis=1)
    
    
    df["RootCN"] = 45
    df["SoilCN"] = 13
    df["RootWt"] = 1000
    df["KDul"] = 0.1
    df["PSIDul"] = -300
    df["VC"] = True
    df["DTmin"] = 0
    df["DTmax"] = 1440
    df["MaxWaterIncrement"] = 10
    df["SpaceWeightingFactor"] = 0
    df["SoluteSpaceWeightingFactor"] = 0
    df["Diagnostics"] = False
    df["Dis"] = 15
    df["Disp"] = 1
    df["A"] = 1
    df["DTHC"] = 0
    df["DTHP"] = 1
    df["WaterTableCl"] = 0
    df["WaterTableNO3"] = 0
    df["WaterTableNH4"] = 0
    df["WaterTableUrea"] = 0
    df["WaterTableTracer"] = 0
    df["WaterTableMineralisationInhibitor"] = 0
    df["WaterTableUreaseInhibitor"] = 0
    df["WaterTableNitrificationInhibitor"] = 0
    df["WaterTableDenitrificationInhibitor"] = 0
    #swim Thickness$double  <- 1000
    df["NO3Exco"] = 0
    df["NO3FIP"] =  1
    df["NH4Exco"] = 100
    df["NH4FIP"] = 1
    df["UreaExco"] = 0
    df["UreaFIP"] = 1
    df["ClExco"] = 0
    df["ClFIP"] = 1
    #df["DrainDepth"] = drainage_parms$DrainDepth
    #horizon$DrainSpacing  <- drainage_parms$DrainSpacing
    #horizon$DrainRadius  <- drainage_parms$DrainRadius
    #horizon$Klat  <- drainage_parms$Klat
    #df["ImpermDepth"] = max(df["bottom"])*10 + 500 
    #df["WaterTableDepth"]  <- df["watertable_g"]*10
    df.to_csv("/Users/timreeves/Desktop/test.csv")
    print(df.head())
Soil_Physical_Prop()

   Unnamed: 0                        texdesc  \
0          13             Very gravelly clay   
1          14  Very gravelly sandy clay loam   
2          15       Very gravelly sandy clay   
3          16        Very gravelly clay loam   
4          17                      Silt loam   

                                          taxclname  cec7  texture  \
0  Fine, mixed, active, thermic Udertic Paleustalfs  22.0    GRV-C   
1  Fine, mixed, active, thermic Udertic Paleustalfs  22.0  GRV-SCL   
2  Fine, mixed, active, thermic Udertic Paleustalfs  22.0   GRV-SC   
3  Fine, mixed, active, thermic Udertic Paleustalfs  22.0   GRV-CL   
4  Fine, mixed, active, thermic Udertic Paleustalfs  10.5      SIL   

         ksat  claytotal  ptotal  hzthk  dbovendry  ...  \
0   28.255389       40.5     NaN   58.0       2.01  ...   
1   28.255389       40.5     NaN   58.0       2.01  ...   
2   28.255389       40.5     NaN   58.0       2.01  ...   
3   28.255389       40.5     NaN   58.0       2.01  ..

In [25]:
df["FBiom"]

0     0.01000
1     0.01000
2     0.01000
3     0.01000
4     0.40000
5     0.01000
6     0.01000
7     0.01000
8     0.01000
9     0.01000
10    0.01000
11    0.02800
12    0.01000
13    0.01000
14    0.01000
15    0.01000
16    0.01000
17    0.01000
18    0.01000
19    0.01000
20    0.01000
21    0.01340
22    0.01340
23    0.01340
24    0.40000
25    0.01350
26    0.01350
27    0.01000
28    0.01000
29    0.01000
       ...   
54    0.01450
55    0.01000
56    0.03250
57    0.01000
58    0.01000
59    0.01000
60    0.01000
61    0.01000
62    0.01040
63    0.01040
64    0.02800
65    0.01000
66    0.01000
67    0.01200
68    0.01000
69    0.01000
70    0.01000
71    0.40000
72    0.01000
73    0.01000
74    0.40000
75    0.01000
76    0.01000
77    0.01000
78    0.01000
79    0.01000
80    0.01000
81    0.01725
82    0.01725
83    0.01725
Name: FBiom, Length: 84, dtype: float64

In [2]:
soil_dir="../data/Soil_data/"
os.system("mkdir "+soil_dir+"tmp_soil/")
#load county coordinate data from working phenotype table (produced in script 1)
soil_areas = pd.read_csv("../data/Phenotype_data/work_set_yields_by_county.csv", index_col=0)
soil_areas.drop_duplicates(subset="ID", inplace=True)
soil_areas = soil_areas[["ID","County","State","Latitude","Longitude"]]
soil_areas.dropna(inplace=True)
#place random county from each state at the top of the list so that one county per state is run first as a test
soil_areas = soil_areas.sample(frac=1, random_state=54321)
soil_areas["tmp"] = soil_areas.duplicated("State")
soil_areas = soil_areas.sort_values("tmp").reset_index(drop=True)
#for the moment we only care abotu downloading entire countys (in most cases the same thing as a single 
#SSURGO soil survey area). For that reason we will use an area of 0.001 degreas in all directions around
#the provided county center data. This is just an area which we are confident lies within the county and
#is used to pull down the data for the area of the entire county (assumed here to be the same as the
#soil survey area)
degs_to_add=0.001
soil_areas["North"] = soil_areas["Latitude"]+degs_to_add
soil_areas["South"] = soil_areas["Latitude"]-degs_to_add
soil_areas["East"] = soil_areas["Longitude"]+degs_to_add
soil_areas["West"] = soil_areas["Longitude"]-degs_to_add

#CSV Patrick made containing names of important soil categories
#TabRead = pd.read_csv("../data/General/TableColumnDescriptionsReport.csv", encoding="ISO-8859-1", skiprows=1)
TabRead = pd.read_csv("../data/General/Revised Table Selections Soil Table.csv", encoding="ISO-8859-1", skiprows=1)
#remove tables that we don't care about
TabRead = TabRead[TabRead["Usable"]=="yes"]
desired_tables = TabRead["Table Physical Name"].unique().tolist()
#import SSURGO table connections
table_relationships = pd.read_csv("../data/General/SSURGO_table_relationships.csv")

In [3]:
failed = {}
all_processed_soils=[]
tablesHeaders = get_table_headers()
count=0
for ID in soil_areas["ID"].unique()[:3]:
    count+=1
    print(count,"/",len(soil_areas["ID"].unique()), ID)
    #Get SSURGO inventory data for desired area of interest
    try:
        MergedCsv, spatial, tabular, majcomp = retrieve_soil_info_by_ID(ID, soil_dir, soil_areas, table_relationships,
                                                               TabRead, desired_tables, verbose=True)
    except Exception as e:
        print(ID, "failed.  Added to list for future attempts.")
        failed[ID] = str(e)
all_processed_soils = pd.concat(all_processed_soils, sort=True)

1 / 2613 UTAH@JUAB
Querying SSURGO to determine which survey area(s) is/are needed.
https://sdmdataaccess.nrcs.usda.gov/Spatial/SDMNAD83Geographic.wfs?Service=WFS&Version=1.0.0&Request=GetFeature&Typename=SurveyAreaPoly&BBOX=-112.79526,39.70982,-112.79326,39.71182
 to: ../data/Soil_data/tmp_soil/UT617_[2019-09-16].zip
15 mapunit mukey component (95, 24)
20 component cokey chorizon (238, 132)
48 chorizon chkey chtexturegrp (608, 302)
2 / 2613 NORTH_DAKOTA@MORTON
Querying SSURGO to determine which survey area(s) is/are needed.
https://sdmdataaccess.nrcs.usda.gov/Spatial/SDMNAD83Geographic.wfs?Service=WFS&Version=1.0.0&Request=GetFeature&Typename=SurveyAreaPoly&BBOX=-101.28074,46.71278,-101.27874,46.71478
 to: ../data/Soil_data/tmp_soil/ND059_[2019-09-16].zip
15 mapunit mukey component (147, 24)
20 component cokey chorizon (887, 132)
48 chorizon chkey chtexturegrp (3855, 302)
3 / 2613 OKLAHOMA@JOHNSTON
Querying SSURGO to determine which survey area(s) is/are needed.
https://sdmdataaccess.

ValueError: No objects to concatenate